# Issues

If you run into a problem or have a question, try an Internet search first and check the list of known issues below. If neither yields a solution, here are some options.

1) Problems concerning online documentation of *ide49* or examples: In the documentation, click the cat icon near to top right and choose `open issue` or `suggest edit`, as appropriate. 

2) Problems with *ide49*: Raise an issue at https://github.com/iot49/ide49. 

Clearly describe what your problem is including your setup (e.g. what microcontroller you are using), steps to reproduce the issue, error messages, and what behavior you would expect. The clearer you are the more likely it is that someone can help you. 

If you have fixes or improvements consider submitting a pull request.

## Known Problems

### No `esp-idf/idf.py` on aarch64

IDF not released for 64-Bit Linux. See https://github.com/espressif/esp-idf/issues/6432.

### Flashing DFU files not working

Flashing DFU files (stm32) from the IDE is not working. 

### mDNS for `.local` domains not working

#### Workaround

This sample code uses the `host` service (which supports mdns) to lookup the IP address and assigns it to the shell environment variable `OUT`:

In [1]:
%%service host --out OUT
ping -c 2 pi4server.local | awk -F"[()]" '{print $2}'

In [1]:
!echo $OUT

10.39.40.151


Assign the recovered address to a CPython variable:

In [1]:
%%host
out = !echo $OUT
print(out)
%store out

['10.39.40.151']
Stored 'out' (SList)


In [2]:
%connect pico

%store -r out
import sys
print("out = {},  platform = {}".format(out, sys.platform))

Connected to pico @ serial:///dev/ttyUSB0
out = ['10.39.40.151'],  platform = esp32


#### Solution?

* Discussion: https://forums.balena.io/t/dns-resolver-in-container/341650/3

In [2]:
%%service host

whoami

ls /etc/dbus-1/system.d/

### Bluetooth

Requires operating (and even starting!) the container as root.

Quick test: run `bluetoothctl` from the command line.

https://forums.balena.io/t/bluetooth-without-root/350427/3

Appears to be a permissions issue, likely on the host OS file `/etc/dbus-1/system.d/bluetooth.conf`. That partition is mounted read only, although this can be changed for testing:

```
If you just want to play around with some files on the HostOS for testing (won’t persist Host updates, not easily carried over to other devices), you could run mount -o remount,rw / in the HostOS to make the root writable.
```

#### Workaround (experimental)

Service bluetooth, available at http://iot49.local/bluetooth has bluez and bleak installed and **runs as root!** I.d. all code is run as root, including notebook, except explicit `su iot`.